In [10]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached numpy-2.2.6.tar.gz (20.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      + /usr/bin/python3 /tmp/pip-install-rbtx1af_/numpy_42d3007200ce4dc59af8ed59669e7e20/vendored-meson/meson/meson.py setup /tmp/pip-install-rbtx1af_/numpy_42d3007200ce4dc59af8ed59669e7e20 /tmp/pip-install-rbtx1af_/numpy_42d3007200ce4dc59af8ed59669e7e20/.mesonpy-xzau2gmo -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=/tmp/pip-install-rbtx1af_/numpy_42d3007200ce4dc59af8ed59669e7e20/.mesonpy-xzau2gmo/meson-pyt

In [8]:
!pip install matplotlib
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
import opencv as cv

ModuleNotFoundError: No module named 'opencv'